<a href="https://colab.research.google.com/github/SarinaMashreghi/ML-notebooks/blob/main/data%20generation/VAE_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data generation


In [1]:
from sklearn.datasets import make_regression, make_classification

X, y = make_classification(n_samples=100,
                        n_features=20,
                           n_informative=10,
                           n_classes=4)
X, y

(array([[ 3.04568832,  1.06022032,  2.98452842, ...,  2.25459163,
          0.37490371,  0.11365388],
        [ 0.11126699, -1.29623538,  2.3733792 , ...,  0.09795778,
          7.44095509, -1.7163683 ],
        [ 0.28047596,  1.80873656, -0.70817941, ..., -1.68074201,
         -0.05288464, -0.18364771],
        ...,
        [-1.15112226,  1.35532487, -0.34450707, ..., -1.1505098 ,
         -0.1172273 , -1.39551933],
        [-3.96100661,  1.86026363, -1.33405745, ..., -3.7178547 ,
          3.74130245, -0.51571354],
        [ 2.23278225,  3.0583209 , -1.55052425, ...,  1.10930034,
         -3.19958255,  0.50592546]]),
 array([3, 0, 3, 2, 3, 0, 1, 2, 2, 3, 2, 3, 0, 0, 2, 0, 2, 3, 1, 2, 1, 2,
        0, 0, 1, 2, 2, 3, 2, 3, 1, 2, 1, 3, 3, 0, 0, 3, 0, 2, 3, 1, 0, 2,
        3, 3, 0, 1, 0, 3, 1, 0, 1, 2, 1, 2, 0, 3, 0, 2, 0, 1, 3, 0, 1, 3,
        0, 1, 1, 1, 0, 0, 2, 1, 1, 1, 3, 3, 2, 3, 1, 2, 0, 1, 1, 2, 1, 0,
        3, 1, 0, 3, 2, 0, 3, 2, 2, 3, 2, 1]))

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X

array([[ 9.03343450e-01,  1.19734321e+00,  7.58212566e-01, ...,
        -4.61652306e-01, -1.17718977e+00, -1.01547305e+00],
       [ 7.88681896e-01, -1.33750666e+00,  2.23855320e-01, ...,
        -1.11754180e+00,  1.54139591e-01, -9.88168872e-01],
       [-3.97578360e-01, -6.17917460e-01,  1.59457837e+00, ...,
         1.25861387e+00, -1.10149099e+00, -1.60370814e+00],
       ...,
       [ 1.16234724e+00,  5.60863726e-04, -5.58074696e-01, ...,
         3.44167460e-01, -3.43355864e-03, -1.17679927e-01],
       [ 4.77316180e-02,  2.27183169e+00, -8.93315540e-01, ...,
        -1.17951343e+00,  8.29357367e-01,  6.05227185e-01],
       [-1.32051649e-02, -3.71768382e-01, -1.27098959e+00, ...,
        -6.71765100e-01,  9.46791938e-02, -2.55242022e-02]])

##VAE

In [2]:
from keras import backend as K
import tensorflow as tf

latent_dim = 10
sample_dim = X.shape[1]

class Sampling(tf.keras.layers.Layer):
  def call(self, inputs):
   mean, log_var = inputs
   return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2)+mean

In [4]:
from tensorflow.keras.layers import *
K.clear_session()

inputs = Input(shape=(sample_dim,))
x = Dense(64, activation="relu")(inputs)
x = Dense(32, activation="relu")(x)

latent_mean = Dense(latent_dim)(x)
latent_var = Dense(latent_dim)(x)

sampling = Sampling()([latent_mean, latent_var])

encoder = tf.keras.Model(inputs=inputs, outputs=[latent_mean, latent_var, sampling], name="encoder")

encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           1344        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 32)           2080        ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 10)           330         ['dense_1[0][0]']                
                                                                                            

In [5]:
decoder_input = Input(shape=(latent_dim,))
x = Dense(32, activation="relu")(decoder_input)
x = Dense(64, activation="relu")(x)
outputs = Dense(sample_dim, "tanh")(x)

decoder = tf.keras.models.Model(inputs=decoder_input, outputs=outputs, name="decoder")

decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense_4 (Dense)             (None, 32)                352       
                                                                 
 dense_5 (Dense)             (None, 64)                2112      
                                                                 
 dense_6 (Dense)             (None, 20)                1300      
                                                                 
Total params: 3,764
Trainable params: 3,764
Non-trainable params: 0
_________________________________________________________________


In [ ]:
vae.compile(loss="mse", optimizer="adam", metrics=["mse"])

vae.fit(X, X, epochs=10)

Epoch 1/10
4/4 [==============================] - 1s 5ms/step - loss: 1.0763 - mse: 1.0763
Epoch 2/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0882 - mse: 1.0882
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0344 - mse: 1.0344
Epoch 4/10
4/4 [==============================] - 0s 3ms/step - loss: 1.0314 - mse: 1.0314
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0180 - mse: 1.0180
Epoch 6/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0196 - mse: 1.0196
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0107 - mse: 1.0107
Epoch 8/10
4/4 [==============================] - 0s 3ms/step - loss: 0.9854 - mse: 0.9854
Epoch 9/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0019 - mse: 1.0019
Epoch 10/10
4/4 [==============================] - 0s 3ms/step - loss: 0.9881 - mse: 0.9881
